In [274]:
%load_ext autoreload
%autoreload 2

from utils import *
import numpy as np
import pandas as pd
import sys
sys.path.append("..")

from torch.utils import data

import glob
import random
import os
import json
import time
import matplotlib.pyplot as plt
import seaborn as sns
from io import StringIO

from bokeh.io import push_notebook, show, output_notebook,reset_output
from bokeh.plotting import figure
import networkx as nx
from bokeh.plotting import figure, from_networkx, curdoc
from bokeh.models import Rect, HoverTool,  TextInput, GraphRenderer, Circle
from bokeh.io import output_file
from bokeh.plotting import figure, output_file, save
from bokeh.models import (BoxZoomTool, Circle, Line, HoverTool,
                          MultiLine, Plot, Range1d, ResetTool)
from bokeh.models import LinearColorMapper, BasicTicker, ColorBar,CustomJSHover,CustomJS
from bokeh.models import Ellipse, GraphRenderer, StaticLayoutProvider

import grandalf

output_notebook()

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Loading BokehJS ...

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [276]:
#RESDIR = "/Users/pari/prism-testbed/test-perflogs/i-0b72d0c3a9125c004/results/perf/"
#LOG = "iozone_-A.csv"
#FN = RESDIR + LOG

RESDIR = "/Users/pari/prism-testbed/LatencyCollectorResults/linux/"
#TAG1 = "exp7"
TAG1 = "exp6"

LT_FN = "lt_instances.txt"
LT_TYPES = '''a1_large_mag_4g=lt-0d15fb8f5bbe9a27d
a1_large_gp3_4g=lt-04840b55d3f795395
r7g_large_gp2_16g=lt-0212ec953ba35b176
t3_large_gp2_8g=lt-05d2d354bc3dd9133
c5a_large_mag_4g=lt-03218e9e27718bbbe
m6a_large_mag_8g=lt-0f6f46002652f9a4c
t3a_medium_gp3_4g=lt-0af65294350b1a8c1
t3a_large_gp3_8g=lt-084bfbae110d52d4e
r6a_large_mag_16g=lt-0e608666ff3adff07
t4g_large_mag_8g=lt-04e0b4826c63bfadb
c7g_large_mag_4g=lt-0af47c6caa3b53b8b
r7g_medium_gp2_16g=lt-01d0081183a7d79f2
t3_xlarge_gp2_16g=lt-0b413bcc22b3ac8fb
'''

LT_TYPES_DF = pd.read_csv(StringIO(LT_TYPES), sep="=", header=None, 
                       names=["lt_type", "lt"])

In [277]:
def load_sys_logs(inp_dir):

    # logfns = glob.iglob(inp_dir + "/*/results/sar_logs*")
    logfns = glob.iglob(inp_dir + "/results/sar_logs*")
    logdfs = {}

    for fi, fn in enumerate(logfns):
        #print(fn)
        if ".csv" in fn or "sar_logs00" in fn:
            continue
        try:
            curdf = pd.read_csv(fn, delimiter=";")
        except Exception as e:
            print(e)
            continue
        if len(curdf) == 0:
            continue

        ts_reps = curdf.groupby("timestamp")["interval"].count()\
                .reset_index().sort_values(by="interval",
                ascending=False)["interval"].values[0]

        if ts_reps > 1:
            curdf = curdf.groupby("timestamp").mean().reset_index()
            curdf = curdf.drop(columns=["interval"])
        else:
            curdf = curdf.drop(columns=["# hostname", "interval"])

        if 'kbmemfree' in curdf.keys():
            logdfs["mem"] = curdf
        elif "dropd/s" in curdf.keys():
            logdfs["network1"] = curdf
        elif "%sio-10" in curdf.keys():
            #system load and pressure-stall statistics
            logdfs["pressure_io"] = curdf
        elif 'kbhugfree' in curdf.keys():
            logdfs["hugepg"] = curdf
        elif '%smem-10' in curdf.keys():
            logdfs["pressure_mem"] = curdf
        elif 'kbswpfree' in curdf.keys():
            logdfs["swap"] = curdf
        elif 'idgm6/s' in curdf.keys():
            logdfs["network2"] = curdf
        elif 'bdscd/s' in curdf.keys():
            logdfs["io1"] = curdf
        elif '%irq' in curdf.keys():
            logdfs["cpu_utilization"] = curdf
        elif 'atmptf/s' in curdf.keys():
            logdfs["network3"] = curdf
        elif 'idgm/s' in curdf.keys():
            logdfs["network4"] = curdf
        elif 'areq-sz' in curdf.keys():
            logdfs["device_io"] = curdf
        elif 'call/s' in curdf.keys():
            logdfs["network_nfs"] = curdf
        elif 'rxdrop/s' in curdf.keys():
            logdfs["network5"] = curdf
        elif 'rxcmp/s' in curdf.keys():
            logdfs["network6"] = curdf
        elif 'tcp6sck' in curdf.keys():
            logdfs["network7"] = curdf
        elif 'iseg/s' in curdf.keys():
            logdfs["network8"] = curdf
        elif 'pswpin/s' in curdf.keys():
            logdfs["swap"] = curdf
        elif 'intr/s' in curdf.keys():
            logdfs["interrupts"] = curdf
        elif '%scpu-10' in curdf.keys():
            logdfs["pressure_cpu"] = curdf
        elif 'MHz' in curdf.keys():
            logdfs["power-cpu"] = curdf
        elif 'degC' in curdf.keys():
            logdfs["power-temp"] = curdf
        elif '%ufsused' in curdf.keys():
            logdfs["filesystem"] = curdf
        elif 'cswch/s' in curdf.keys():
            logdfs["context_switch"] = curdf
        elif 'pgpgin/s' in curdf.keys():
            logdfs["paging"] = curdf
        elif 'fwddgm/s' in curdf.keys():
            logdfs["network9"] = curdf
        elif 'ihdrer6/s' in curdf.keys():
            logdfs["network10"] = curdf
        elif 'imsg/s' in curdf.keys():
            logdfs["network11"] = curdf
        elif 'ierr6/s' in curdf.keys():
            logdfs["network12"] = curdf
        elif 'scall/s' in curdf.keys():
            logdfs["network13"] = curdf
        elif 'runq-sz' in curdf.keys():
            logdfs["pressure_load"] = curdf
        elif 'dentunusd' in curdf.keys():
            logdfs["inode"] = curdf
        elif 'igmbq6/s' in curdf.keys():
            logdfs["network14"] = curdf
        elif 'tcpsck' in curdf.keys():
            logdfs["network15"] = curdf
        elif 'otmex/s' in curdf.keys():
            logdfs["network16"] = curdf
        elif 'ihdrerr/s' in curdf.keys():
            logdfs["network17"] = curdf
        elif 'irec6/s' in curdf.keys():
            logdfs["network18"] = curdf
        else:
            assert False

    keys = list(logdfs.keys())
    for k in keys:
        if "network" in k:
            del logdfs[k]
            continue
        if k in ["power-cpu", "hugepg", "power-temp"]:
            del logdfs[k]
            continue

    keys = list(logdfs.keys())
    if len(keys) == 0:
        return []

    curdf = logdfs[keys[0]]

    for ki in range(1,len(keys)):
        curdf = curdf.merge(logdfs[keys[ki]], on="timestamp")

    return curdf

PERF_NAMES = ["value", "unit", "stat_name", "job_time", "util?", "value2", "unit2"]

def load_all_logs(inp_tag, inp_dir, skip_timeouts=False):
    inp_dir = os.path.join(inp_dir, inp_tag)
    if not os.path.exists(inp_dir):
        print("no exists")
        return [],[]

    dfs = []
    all_logs = {}

    instance_dirs = os.listdir(inp_dir)
    lt_fn_path = os.path.join(inp_dir, LT_FN)
    if os.path.exists(lt_fn_path):
        ltdf = pd.read_csv(lt_fn_path, header=None,
                   names=["instance", "lt"])
        ltdf = ltdf.merge(LT_TYPES_DF, on="lt")
    else:
        print("lt fn path doesn't exist")
        return [],[]
    
    print(instance_dirs)
    for iname in instance_dirs:
        curdir = os.path.join(inp_dir, iname)
        if not os.path.isdir(curdir):
            continue
          
        perfcsvs = glob.iglob(curdir + "/results/perf/*.csv")
        cmds_fn = curdir + "/results/perf/allcommands.csv"
        
        print(cmds_fn)
        cmdsdf = pd.read_csv(cmds_fn, 
                             names=["jobhash", "cmd", "fn", "start_time","exec_time", "status"], 
                             error_bad_lines=False,
                             header=None)
        
        
        curdfs = []
        for pfn in perfcsvs:
            hashcsv = os.path.basename(pfn)
            jobhash = hashcsv.replace(".csv", "")
            
            if "allcommands" in pfn:
                continue
            try:
                currt = pd.read_csv(pfn, skiprows=1, 
                                    names = PERF_NAMES,
                                    header=None)
                #print(currt)
            except Exception as e:
                print(e)
                continue
            if len(currt) == 0:
                continue
                
            currt["jobhash"] = jobhash
            #currt["instance"] = iname
            curdfs.append(currt)

        if len(curdfs) == 0:
            continue

        currt = pd.concat(curdfs)
        if len(currt) == 0:
            continue
        
        #print(currt)
        #currt = currt.merge(ltdf, on="instance")
        cmdsdf["instance"] = iname
        cmdsdf = cmdsdf.merge(ltdf, on="instance")
        currt = currt.merge(cmdsdf, on="jobhash", how="right")

        if len(currt) == 0:
            print(iname, "rt == 0")
            continue

        curlogs = load_sys_logs(curdir)

        # skip queries which are outside logged time
        if len(curlogs) == 0:
            continue

        curlogs = curlogs.dropna()
        if len(curlogs) == 0:
            continue

        # remove part of logs which aren't in currts
        # max_query_start = max(currt["start_time"].values) + 60.0
        # min_query_start = min(currt["start_time"].values) - 650.0
        # curlogs = curlogs[curlogs["timestamp"] > min_query_start]
        # curlogs = curlogs[curlogs["timestamp"] < max_query_start]

        if len(curlogs) == 0:
            print("curlogs == 0 after filtering for: ", inp_tag)
            continue

        if len(currt) == 0:
            continue

        dfs.append(currt)
        all_logs[iname] = curlogs

    if len(dfs) == 0:
        return [],[]

    df = pd.concat(dfs)

    if len(all_logs) == 0:
        return [],[]

    return df, all_logs

In [278]:
df,syslogs = load_all_logs(TAG1, RESDIR)
# df2,syslogs2 = load_all_logs(TAG2, RESDIR)
# df = pd.concat([df,df2])

['i-0c3e218923ef8e480', 'lt_instances.txt', 'i-0ebf5fb112e217038', 'i-0d178748decd988b0', 'i-02f1a742c193d5877', 'i-0a6675d75bb220943', 'i-05eec3754bddc984b', 'i-02a38759915e78457', 'i-094bc37cf0b6ade69', 'i-0c529d5234191135e', 'i-04fce02a907ad201a', 'i-0628d14a006f2edd3']
/Users/pari/prism-testbed/LatencyCollectorResults/linux/exp6/i-0c3e218923ef8e480/results/perf/allcommands.csv


/var/folders/kq/_m01bm_55_b60jjn3jx066p80000gn/T/ipykernel_93582/382273177.py:158: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  cmdsdf = pd.read_csv(cmds_fn,
/var/folders/kq/_m01bm_55_b60jjn3jx066p80000gn/T/ipykernel_93582/382273177.py:24: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  curdf = curdf.groupby("timestamp").mean().reset_index()
/var/folders/kq/_m01bm_55_b60jjn3jx066p80000gn/T/ipykernel_93582/382273177.py:24: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  curdf = curdf.groupby("timestamp").mean().reset_

/Users/pari/prism-testbed/LatencyCollectorResults/linux/exp6/i-0ebf5fb112e217038/results/perf/allcommands.csv


/var/folders/kq/_m01bm_55_b60jjn3jx066p80000gn/T/ipykernel_93582/382273177.py:158: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  cmdsdf = pd.read_csv(cmds_fn,
/var/folders/kq/_m01bm_55_b60jjn3jx066p80000gn/T/ipykernel_93582/382273177.py:24: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  curdf = curdf.groupby("timestamp").mean().reset_index()
/var/folders/kq/_m01bm_55_b60jjn3jx066p80000gn/T/ipykernel_93582/382273177.py:24: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  curdf = curdf.groupby("timestamp").mean().reset_

/Users/pari/prism-testbed/LatencyCollectorResults/linux/exp6/i-0d178748decd988b0/results/perf/allcommands.csv


...

/var/folders/kq/_m01bm_55_b60jjn3jx066p80000gn/T/ipykernel_93582/382273177.py:158: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  cmdsdf = pd.read_csv(cmds_fn,
/var/folders/kq/_m01bm_55_b60jjn3jx066p80000gn/T/ipykernel_93582/382273177.py:24: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  curdf = curdf.groupby("timestamp").mean().reset_index()
/var/folders/kq/_m01bm_55_b60jjn3jx066p80000gn/T/ipykernel_93582/382273177.py:24: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  curdf = curdf.groupby("timestamp").mean().reset_

/Users/pari/prism-testbed/LatencyCollectorResults/linux/exp6/i-04fce02a907ad201a/results/perf/allcommands.csv


/var/folders/kq/_m01bm_55_b60jjn3jx066p80000gn/T/ipykernel_93582/382273177.py:158: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  cmdsdf = pd.read_csv(cmds_fn,
/var/folders/kq/_m01bm_55_b60jjn3jx066p80000gn/T/ipykernel_93582/382273177.py:24: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  curdf = curdf.groupby("timestamp").mean().reset_index()
/var/folders/kq/_m01bm_55_b60jjn3jx066p80000gn/T/ipykernel_93582/382273177.py:24: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  curdf = curdf.groupby("timestamp").mean().reset_

/Users/pari/prism-testbed/LatencyCollectorResults/linux/exp6/i-0628d14a006f2edd3/results/perf/allcommands.csv


/var/folders/kq/_m01bm_55_b60jjn3jx066p80000gn/T/ipykernel_93582/382273177.py:158: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  cmdsdf = pd.read_csv(cmds_fn,
/var/folders/kq/_m01bm_55_b60jjn3jx066p80000gn/T/ipykernel_93582/382273177.py:24: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  curdf = curdf.groupby("timestamp").mean().reset_index()
/var/folders/kq/_m01bm_55_b60jjn3jx066p80000gn/T/ipykernel_93582/382273177.py:24: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  curdf = curdf.groupby("timestamp").mean().reset_

In [279]:
df.groupby(["status"])["status"].count()

status
0      15168
1       8060
124      396
Name: status, dtype: int64

In [280]:
df.head(60)

,value,unit,stat_name,job_time,util?,value2,unit2,jobhash,cmd,fn,start_time,exec_time,status,instance,lt,lt_type
0,12701,NaN,cycles,32460.0,100.0,NaN,NaN,4c4ed904f223f5a4be7f55e71003deba7fced1292da222...,sudo fio --filename=./test --rw=read --ioengin...,newfio.sh,1679952365,25,0,i-0c3e218923ef8e480,lt-04e0b4826c63bfadb,t4g_large_mag_8g
1,<not supported>,NaN,instructions,0.0,100.0,NaN,NaN,4c4ed904f223f5a4be7f55e71003deba7fced1292da222...,sudo fio --filename=./test --rw=read --ioengin...,newfio.sh,1679952365,25,0,i-0c3e218923ef8e480,lt-04e0b4826c63bfadb,t4g_large_mag_8g
2,<not supported>,NaN,cache-misses,0.0,100.0,NaN,NaN,4c4ed904f223f5a4be7f55e71003deba7fced1292da222...,sudo fio --filename=./test --rw=read --ioengin...,newfio.sh,1679952365,25,0,i-0c3e218923ef8e480,lt-04e0b4826c63bfadb,t4g_large_mag_8g
3,0,NaN,page-faults,14359.0,100.0,NaN,NaN,4c4ed904f223f5a4be7f55e71003deba7fced1292da222...,sudo fio --filename=./test --rw=read --ioengin...,newfio.sh,1679952365,25,0,i-0c3e218923ef8e480,lt-04e0b4826c63bfadb,t4g_large_mag_8g
4,<not supported>,NaN,branch-misses,0.0,100.0,NaN,NaN,4c4ed904f223f5a4be7f55e71003deba7fced1292da222...,sudo fio --filename=./test --rw=read --ioengin...,newfio.sh,1679952365,25,0,i-0c3e218923ef8e480,lt-04e0b4826c63bfadb,t4g_large_mag_8g
5,0,NaN,context-switches,14359.0,100.0,NaN,NaN,4c4ed904f223f5a4be7f55e71003deba7fced1292da222...,sudo fio --filename=./test --rw=read --ioengin...,newfio.sh,1679952365,25,0,i-0c3e218923ef8e480,lt-04e0b4826c63bfadb,t4g_large_mag_8g
6,<not supported>,NaN,L1-dcache-load-misses,0.0,100.0,NaN,NaN,4c4ed904f223f5a4be7f55e71003deba7fced1292da222...,sudo fio --filename=./test --rw=read --ioengin...,newfio.sh,1679952365,25,0,i-0c3e218923ef8e480,lt-04e0b4826c63bfadb,t4g_large_mag_8g
7,9854,NaN,cycles,29809.0,100.0,NaN,NaN,49a0426469b87bfcbb19f327dd14136ca51db5e4b41c1b...,sudo fio --filename=./test --rw=read --ioengin...,newfio.sh,1679952391,22,0,i-0c3e218923ef8e480,lt-04e0b4826c63bfadb,t4g_large_mag_8g
8,<not supported>,NaN,instructions,0.0,100.0,NaN,NaN,49a0426469b87bfcbb19f327dd14136ca51db5e4b41c1b...,sudo fio --filename=./test --rw=read --ioengin...,newfio.sh,1679952391,22,0,i-0c3e218923ef8e480,lt-04e0b4826c63bfadb,t4g_large_mag_8g
9,<not supported>,NaN,cache-misses,0.0,100.0,NaN,NaN,49a0426469b87bfcbb19f327dd14136ca51db5e4b41c1b...,sudo fio --filename=./test --rw=read --ioengin...,newfio.sh,1679952391,22,0,i-0c3e218923ef8e480,lt-04e0b4826c63bfadb,t4g_large_mag_8g


In [281]:
df = df[df["status"].isin([0, 124])]
#df = df[df["exec_time"] >= 1.0]

In [282]:
print(set(df["status"]))

{0, 124}


# AWS arm instances support hardware perf-stat events and intel ones don't?

In [283]:
# df = df[df["value"] != "<not supported>"]
# df = df[df["value"] != "<not counted>"]
df.groupby(["lt_type", "stat_name"])["value"].describe()

count unique              top freq
lt_type            stat_name                                               
a1_large_gp3_4g    L1-dcache-load-misses   398      1    <not counted>  398
                   branch-misses           398     43              114   46
                   cache-misses            398     56              109   25
                   context-switches        398      2                0  397
                   cycles                  398    310            14746    4
                   instructions            398     21             3957  372
                   page-faults             398      1                0  398
c5a_large_mag_4g   L1-dcache-load-misses   166      1  <not supported>  166
                   branch-misses           166      1  <not supported>  166
                   cache-misses            166      1  <not supported>  166
                   context-switches        166      1                0  166
                   cycles                  166      1  <not supported>  166
                   instructions            166      1  <not supported>  166
                   page-faults             166      1                0  166
c7g_large_mag_4g   L1-dcache-load-misses   188      1    <not counted>  188
                   branch-misses           188      1    <not counted>  188
                   cache-misses            188     40              154   24
                   context-switches        188      1                0  188
                   cycles                  188    145             8682    3
                   instructions            188      7             3832  181
                   page-faults             188      1                0  188
m6a_large_mag_8g   L1-dcache-load-misses   133      1  <not supported>  133
                   branch-misses           133      1  <not supported>  133
                   cache-misses            133      1  <not supported>  133
                   context-switches        133      1                0  133
                   cycles                  133      1  <not supported>  133
                   instructions            133      1  <not supported>  133
                   page-faults             133      1                0  133
r6a_large_mag_16g  L1-dcache-load-misses   167      1  <not supported>  167
                   branch-misses           167      1  <not supported>  167
                   cache-misses            167      1  <not supported>  167
                   context-switches        167      1                0  167
                   cycles                  167      1  <not supported>  167
                   instructions            167      1  <not supported>  167
                   page-faults             167      1                0  167
r7g_large_gp2_16g  L1-dcache-load-misses   148      1    <not counted>  148
                   branch-misses           148      1    <not counted>  148
                   cache-misses            148     32              159   13
                   context-switches        148      1                0  148
                   cycles                  148    113             8055    3
                   instructions            148      7             3832  140
                   page-faults             148      1                0  148
r7g_medium_gp2_16g L1-dcache-load-misses   137      1    <not counted>  137
                   branch-misses           137      1    <not counted>  137
                   cache-misses            137     25              124   17
                   context-switches        137      2                0  135
                   cycles                  137    108             7587    3
                   instructions            137      5             3832  132
                   page-faults             137      1                0  137
t3_large_gp2_8g    L1-dcache-load-misses   132      1  <not supported>  132
                   branch-misses           132      1  <not supported>  132
             

In [284]:
len(set(df["cmd"]))

169

In [285]:
# tmp = df[df["lt_type"] == "r6a_large_mag_16g"]
# #tmp = tmp[tmp["cmd"].str.contains("ffmpeg")]
# tmp

In [286]:
print(set(df["stat_name"]))
print(set(df["status"]))

{nan, 'L1-dcache-load-misses', 'cache-misses', 'context-switches', 'cycles', 'instructions', 'page-faults', 'branch-misses'}
{0, 124}


# CGrep

In [289]:
pd.set_option('display.max_rows', 1000)
#tmp = df[df["stat_name"] == "page-faults"]
#tmp = df
#tmp["value"] = tmp["value"].astype("float")

tmp = df.drop_duplicates("jobhash")
tmp = tmp[tmp["cmd"].str.contains("fio")]
#tmp.groupby(["cmd","lt_type"])["value"].describe()

print(set(tmp["status"]))
tmp.groupby(["lt_type"])["exec_time"].describe().round(2)

#tmp.groupby(["lt_type"])["job_time"].describe()

{0, 124}


,count,mean,std,min,25%,50%,75%,max
lt_type,,,,,,,,
a1_large_gp3_4g,326.0,128.61,203.37,1.0,14.00,47.0,132.75,900.0
c5a_large_mag_4g,162.0,279.56,356.92,0.0,20.00,81.0,449.00,901.0
c7g_large_mag_4g,175.0,256.45,349.37,0.0,11.00,45.0,436.00,900.0
m6a_large_mag_8g,134.0,328.83,346.65,0.0,40.50,149.5,517.75,900.0
r6a_large_mag_16g,165.0,272.34,357.47,0.0,12.00,50.0,453.00,901.0
r7g_large_gp2_16g,156.0,293.56,383.25,0.0,6.75,69.0,900.00,901.0
r7g_medium_gp2_16g,150.0,293.13,387.77,0.0,8.00,46.0,900.00,900.0
t3_large_gp2_8g,148.0,302.99,396.74,0.0,8.75,47.0,900.00,901.0
t3_xlarge_gp2_16g,152.0,288.89,385.37,1.0,8.00,57.5,900.00,900.0


In [243]:
pd.set_option('display.max_rows', 1000)
#tmp = df[df["stat_name"] == "page-faults"]
#tmp = df
#tmp["value"] = tmp["value"].astype("float")

#tmp = df.drop_duplicates("jobhash")
tmp = df[df["status"] == 124]
#tmp = tmp[tmp["cmd"].str.contains("psql")]
#tmp.groupby(["cmd","lt_type"])["value"].describe()

print(set(tmp["status"]))
tmp.groupby(["lt_type"])["exec_time"].describe()

#tmp.groupby(["lt_type"])["job_time"].describe()

{124}


ValueError: index must be a MultiIndex to unstack, <class 'pandas.core.indexes.base.Index'> was passed

In [207]:
tmp.groupby(["cmd", "lt_type"]).count().head(50)

value  \
cmd                                                lt_type                     
psql -d imdb -U ubuntu -p 5432 < ../../queries/... a1_large_gp3_4g         1   
                                                   m6a_large_mag_8g        2   
                                                   r6a_large_mag_16g       4   
                                                   r7g_large_gp2_16g       3   
                                                   r7g_medium_gp2_16g      3   
                                                   t3_large_gp2_8g         3   
                                                   t3_xlarge_gp2_16g       3   
                                                   t3a_medium_gp3_4g       2   
                                                   t4g_large_mag_8g        2   
psql -d imdb -U ubuntu -p 5432 < ../../queries/... a1_large_gp3_4g         1   
                                                   m6a_large_mag_8g        2   
                                                   r6a_large_mag_16g       4   
                                                   r7g_large_gp2_16g       3   
                                                   r7g_medium_gp2_16g      3   
                                                   t3_large_gp2_8g         3   
                                                   t3_xlarge_gp2_16g       3   
                                                   t3a_medium_gp3_4g       2   
                                                   t4g_large_mag_8g        2   
psql -d imdb -U ubuntu -p 5432 < ../../queries/... a1_large_gp3_4g         1   
                                                   m6a_large_mag_8g        2   
                                                   r6a_large_mag_16g       4   
                                                   r7g_large_gp2_16g       3   
                                                   r7g_medium_gp2_16g      3   
                                                   t3_large_gp2_8g         3   
                                                   t3_xlarge_gp2_16g       3   
                                                   t3a_medium_gp3_4g       2   
                                                   t4g_large_mag_8g        2   
psql -d imdb -U ubuntu -p 5432 < ../../queries/... a1_large_gp3_4g         1   
                                                   m6a_large_mag_8g        2   
                                                   r6a_large_mag_16g       4   
                                                   r7g_large_gp2_16g       3   
                                                   r7g_medium_gp2_16g      3   
                                                   t3_large_gp2_8g         3   
                                                   t3_xlarge_gp2_16g       3   
                                                   t3a_medium_gp3_4g       2   
                                                   t4g_large_mag_8g        2   
psql -d imdb -U ubuntu -p 5432 < ../../queries/... a1_large_gp3_4g         1   
                                                   m6a_large_mag_8g        2   
                                                   r6a_large_mag_16g       4   
                                                   r7g_large_gp2_16g       3   
                                                   r7g_medium_gp2_16g      3   
                                                   t3_large_gp2_8g         3   
                                                   t3_xlarge_gp2_16g       3   
                                                   t3a_medium_gp3_4g       2   
                                                   t4g_large_mag_8g        2   
psql -d imdb -U ubuntu -p 5432 < ../../queries/... a1_large_gp3_4g         1   
                                                   m6a_large_mag_8g        2   
                                                   r6a_large_mag_16g       4   
                                                   r7g_large_gp2_16g   

# Memtier benchmarks

In [59]:
pd.set_option('display.max_rows', 1000)
tmp = df[df["stat_name"] == "page-faults"]
#tmp = df
tmp["value"] = tmp["value"].astype("float")
tmp = tmp[tmp["cmd"].str.contains("memtier")]
#tmp.groupby(["cmd","lt_type"])["value"].describe()
tmp.groupby(["cmd", "lt_type"])["exec_time"].describe()

/var/folders/kq/_m01bm_55_b60jjn3jx066p80000gn/T/ipykernel_93582/4184579574.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp["value"] = tmp["value"].astype("float")


count  \
cmd                                                lt_type                     
memtier_benchmark -R -n 1000 --data-size-range=... a1_large_gp3_4g       1.0   
                                                   c5a_large_mag_4g      3.0   
                                                   c7g_large_mag_4g      3.0   
                                                   m6a_large_mag_8g      3.0   
                                                   r6a_large_mag_16g     3.0   
                                                   r7g_large_gp2_16g     3.0   
                                                   r7g_medium_gp2_16g    3.0   
                                                   t3_large_gp2_8g       3.0   
                                                   t3_xlarge_gp2_16g     3.0   
                                                   t3a_medium_gp3_4g     3.0   
                                                   t4g_large_mag_8g      1.0   
memtier_benchmark -R -n 10000 --data-size-range... a1_large_gp3_4g       1.0   
                                                   c5a_large_mag_4g      3.0   
                                                   c7g_large_mag_4g      3.0   
                                                   m6a_large_mag_8g      3.0   
                                                   r6a_large_mag_16g     3.0   
                                                   r7g_large_gp2_16g     3.0   
                                                   r7g_medium_gp2_16g    3.0   
                                                   t3_large_gp2_8g       3.0   
                                                   t3_xlarge_gp2_16g     3.0   
                                                   t3a_medium_gp3_4g     3.0   
                                                   t4g_large_mag_8g      1.0   
memtier_benchmark -R -n 100000 --data-size-rang... a1_large_gp3_4g       1.0   
                                                   c5a_large_mag_4g      2.0   
                                                   c7g_large_mag_4g      3.0   
                                                   m6a_large_mag_8g      3.0   
                                                   r6a_large_mag_16g     3.0   
                                                   r7g_large_gp2_16g     3.0   
                                                   t3_large_gp2_8g       3.0   
                                                   t3_xlarge_gp2_16g     3.0   
                                                   t3a_medium_gp3_4g     2.0   
                                                   t4g_large_mag_8g      1.0   
memtier_benchmark -R -n 1000000 --data-size-ran... c7g_large_mag_4g      2.0   
                                                   r7g_large_gp2_16g     2.0   
memtier_benchmark -R -n 25000 --data-size-range... c5a_large_mag_4g      1.0   
                                                   c7g_large_mag_4g      1.0   
                                                   m6a_large_mag_8g      1.0   
                                                   r6a_large_mag_16g     1.0   
                                                   r7g_large_gp2_16g     1.0   
                                                   r7g_medium_gp2_16g    1.0   
                                                   t3_large_gp2_8g       1.0   
                                                   t3_xlarge_gp2_16g     1.0   
                                                   t3a_medium_gp3_4g     1.0   
memtier_benchmark -R -n 50000 --data-size-range... c5a_large_mag_4g      1.0   
                                                   c7g_large_mag_4g      1.0   
                                                   m6a_large_mag_8g      1.0   
                                                   r6a_large_mag_16g     1.0   
                                                   r7g_large_gp2_16g     1.0   
                                                   r7g_medium_gp2_16g  

In [52]:
#df.groupby(["lt_type"])["instance"].describe()

In [25]:
#df.groupby(["lt_type", "status"]).count()

In [54]:
tmp = df[df["stat_name"] == "page-faults"]
tmp.groupby(["lt_type"])["exec_time"].describe()

,count,mean,std,min,25%,50%,75%,max
lt_type,,,,,,,,
a1_large_gp3_4g,96.0,120.531250,229.019280,0.0,1.0,11.5,104.00,1356.0
c5a_large_mag_4g,154.0,83.928571,157.421714,0.0,2.0,14.0,90.75,771.0
c7g_large_mag_4g,163.0,84.969325,153.805931,0.0,2.0,16.0,92.50,696.0
m6a_large_mag_8g,162.0,79.320988,141.934490,0.0,2.0,15.0,92.00,696.0
r6a_large_mag_16g,168.0,80.964286,148.779783,0.0,1.0,13.0,92.00,848.0
r7g_large_gp2_16g,190.0,93.115789,168.596407,0.0,1.0,11.5,90.75,779.0
r7g_medium_gp2_16g,157.0,111.770701,184.898859,0.0,1.0,19.0,120.00,924.0
t3_large_gp2_8g,163.0,105.564417,182.250947,0.0,1.0,18.0,92.00,1216.0
t3_xlarge_gp2_16g,157.0,96.853503,185.605551,0.0,2.0,15.0,92.00,1121.0


In [55]:
tmp.groupby(["cmd","lt_type"])["exec_time"].describe()

count  \
cmd                                                lt_type                     
./bench1                                           a1_large_gp3_4g       1.0   
                                                   c5a_large_mag_4g      1.0   
                                                   c7g_large_mag_4g      1.0   
                                                   m6a_large_mag_8g      1.0   
                                                   r6a_large_mag_16g     1.0   
                                                   r7g_large_gp2_16g     2.0   
                                                   r7g_medium_gp2_16g    1.0   
                                                   t3_large_gp2_8g       1.0   
                                                   t3_xlarge_gp2_16g     1.0   
                                                   t3a_medium_gp3_4g     1.0   
                                                   t4g_large_mag_8g      1.0   
./bench2                                           a1_large_gp3_4g       1.0   
                                                   c5a_large_mag_4g      1.0   
                                                   c7g_large_mag_4g      1.0   
                                                   m6a_large_mag_8g      1.0   
                                                   r6a_large_mag_16g     1.0   
                                                   r7g_large_gp2_16g     2.0   
                                                   r7g_medium_gp2_16g    1.0   
                                                   t3_large_gp2_8g       1.0   
                                                   t3_xlarge_gp2_16g     1.0   
                                                   t3a_medium_gp3_4g     1.0   
                                                   t4g_large_mag_8g      1.0   
./bench3                                           a1_large_gp3_4g       1.0   
                                                   c5a_large_mag_4g      1.0   
                                                   c7g_large_mag_4g      1.0   
                                                   m6a_large_mag_8g      1.0   
                                                   r6a_large_mag_16g     1.0   
                                                   r7g_large_gp2_16g     2.0   
                                                   r7g_medium_gp2_16g    1.0   
                                                   t3_large_gp2_8g       1.0   
                                                   t3_xlarge_gp2_16g     1.0   
                                                   t3a_medium_gp3_4g     1.0   
                                                   t4g_large_mag_8g      1.0   
./bench4                                           a1_large_gp3_4g       1.0   
                                                   c5a_large_mag_4g      1.0   
                                                   c7g_large_mag_4g      1.0   
                                                   m6a_large_mag_8g      1.0   
                                                   r6a_large_mag_16g     1.0   
                                                   r7g_large_gp2_16g     2.0   
                                                   r7g_medium_gp2_16g    1.0   
                                                   t3_large_gp2_8g       1.0   
                                                   t3_xlarge_gp2_16g     1.0   
                                                   t3a_medium_gp3_4g     1.0   
                                                   t4g_large_mag_8g      1.0   
./bench5                                           a1_large_gp3_4g       1.0   
                                                   c5a_large_mag_4g      1.0   
                                                   c7g_large_mag_4g      1.0   
                                                   m6a_large_mag_8g      1.0   
                                                   r6a_large_mag_16g   

In [28]:
#df = pd.read_csv(FN, skiprows=1, header=None)

In [30]:
os.path.exists('/Users/pari/prism-testbed/test-perflogs/i-0b72d0c3a9125c004/results/')

True

In [31]:
df.keys()

Int64Index([0, 1, 2, 3, 4, 5, 6], dtype='int64')

In [32]:
df

,0,1,2,3,4,5,6
0,<not supported>,NaN,dTLB-loads,0,100.0,NaN,NaN
1,<not supported>,NaN,iTLB-loads,0,100.0,NaN,NaN
2,<not supported>,NaN,branch-loads,0,100.0,NaN,NaN
3,<not supported>,NaN,instructions,0,100.0,NaN,NaN
4,<not supported>,NaN,cache-references,0,100.0,NaN,NaN
5,28783.96,msec,cpu-clock,28817188873,100.0,0.025,CPUs utilized
6,28817.19,msec,task-clock,28817188873,100.0,0.025,CPUs utilized
7,16521,NaN,page-faults,28817188873,100.0,573.634,/sec
8,16516,NaN,minor-faults,28817188873,100.0,573.461,/sec
9,5,NaN,major-faults,28817188873,100.0,0.174,/sec
